In [21]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import pickle, gzip, numpy
import numpy as np

In [22]:
f = gzip.open('mnist.pkl.gz', 'rb')
train_set, valid_set, test_set = pickle.load(f,encoding='latin1')
f.close()
X_train, y_train = train_set[0], train_set[1]
X_valid, y_valid = valid_set[0], valid_set[1]
X_test,  y_test  = test_set[0],  test_set[1]

def to_one_hot(y, n_classes=10): # You might want to use this as some point...
    _y = np.zeros((len(y), n_classes))
    _y[np.arange(len(y)), y] = 1
    return _y

#reformat
y_train = to_one_hot(y_train, n_classes=10)
y_valid = to_one_hot(y_valid, n_classes=10)
y_test = to_one_hot(y_test, n_classes=10)
print('Training set', X_train.shape, y_train.shape)
print('Validation set', X_valid.shape, y_valid.shape)
print('Test set', X_test.shape, y_test.shape)


Training set (50000, 784) (50000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [23]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [33]:
batch_size = 128
image_size = 28
num_labels = 10

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(X_valid)
  tf_test_dataset = tf.constant(X_test)
  
  # Variables.
  W1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 20]))
  b1 = tf.Variable(tf.zeros([20]))
    
  W2 = tf.Variable(
    tf.truncated_normal([20, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  A1 = tf.matmul(tf_train_dataset, W1) + b1
  A2 = tf.sigmoid(A1)
  Z =  tf.matmul(A2, W2) + b2


  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=Z))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(Z)
  valid_prediction = tf.nn.softmax(
      tf.matmul(tf.sigmoid(tf.matmul(tf_valid_dataset, W1) + b1), W2) + b2)
  test_prediction = tf.nn.softmax(
      tf.matmul(tf.sigmoid(tf.matmul(tf_test_dataset, W1) + b1), W2) + b2)

In [36]:


num_steps = 10000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    offset = (step * batch_size) % (y_train.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = X_train[offset:(offset + batch_size), :]
    batch_labels = y_train[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # key of the dictionary is placeholder node of the graph to be fed,
    # and value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), y_valid))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), y_test))



Initialized
Minibatch loss at step 0: 4.907996
Minibatch accuracy: 5.5%
Validation accuracy: 7.0%
Minibatch loss at step 500: 0.749003
Minibatch accuracy: 78.1%
Validation accuracy: 82.4%
Minibatch loss at step 1000: 0.616335
Minibatch accuracy: 83.6%
Validation accuracy: 86.1%
Minibatch loss at step 1500: 0.732347
Minibatch accuracy: 82.0%
Validation accuracy: 87.7%
Minibatch loss at step 2000: 0.297789
Minibatch accuracy: 90.6%
Validation accuracy: 89.1%
Minibatch loss at step 2500: 0.460202
Minibatch accuracy: 85.9%
Validation accuracy: 89.8%
Minibatch loss at step 3000: 0.193994
Minibatch accuracy: 94.5%
Validation accuracy: 90.4%
Minibatch loss at step 3500: 0.416146
Minibatch accuracy: 88.3%
Validation accuracy: 90.7%
Minibatch loss at step 4000: 0.190437
Minibatch accuracy: 93.0%
Validation accuracy: 91.2%
Minibatch loss at step 4500: 0.322153
Minibatch accuracy: 86.7%
Validation accuracy: 91.8%
Minibatch loss at step 5000: 0.322256
Minibatch accuracy: 89.8%
Validation accuracy:

In [39]:
A2.shape

TensorShape([Dimension(128), Dimension(20)])